# Load Dataset and DataLoader

In [1]:
import os
import torch

AISHELL_TRANSCRIPT_PATH = "data_aishell\\transcript\\aishell_transcript_v0.8.txt"
AISHELL_WAV_ROOT = "data_aishell\\wav" # Thư mục cha chứa train/dev/test
if not os.path.exists(AISHELL_TRANSCRIPT_PATH): exit(f"Không tìm thấy file transcript tại: {AISHELL_TRANSCRIPT_PATH}")
if not os.path.exists(AISHELL_WAV_ROOT): exit(f"Không tìm thấy thư mục wav gốc tại: {AISHELL_WAV_ROOT}")
if not all(os.path.isdir(os.path.join(AISHELL_WAV_ROOT, d)) for d in ['train', 'dev', 'test']):
        print(f"Thiếu một hoặc nhiều thư mục con 'train', 'dev', 'test' bên trong: {AISHELL_WAV_ROOT}")
SAMPLE_RATE = 16000; N_MELS = 80; FRAME_LENGTH = 25; FRAME_SHIFT = 10
VGG_OUT_CHANNELS = 128; BATCH_SIZE = 4; NUM_WORKERS = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Thiếu một hoặc nhiều thư mục con 'train', 'dev', 'test' bên trong: data_aishell\wav


In [6]:
%ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is FAA0-AAE1

 Directory of c:\Users\huy\OneDrive\Desktop\BTTH-DS313

04/09/2025  03:43 PM    <DIR>          .
04/08/2025  03:21 PM    <DIR>          ..
04/08/2025  07:11 PM    <DIR>          asr_model_project
03/18/2025  09:52 AM             6,795 BTTH_Nhom1.ipynb
03/23/2025  10:09 AM         1,522,595 Chapter-4-Speech-Synthesis_2.pdf
04/08/2025  04:09 PM    <DIR>          data_aishell
04/08/2025  03:32 PM            10,482 dataloader_downsampled.py
03/23/2025  10:08 AM           207,129 DS313 HomeWork3.pdf
04/08/2025  03:47 PM           625,945 qian23_interspeech.pdf
04/08/2025  08:17 PM               617 remind.txt
               6 File(s)      2,373,563 bytes
               4 Dir(s)  30,884,810,752 bytes free


In [7]:
from dataloader_downsampled import AISHELL1Dataset, collate_fn
from torch.utils.data import DataLoader

SAMPLE_RATE = 16000; N_MELS = 80; FRAME_LENGTH = 25; FRAME_SHIFT = 10
VGG_OUT_CHANNELS = 128; BATCH_SIZE = 4; NUM_WORKERS = 0
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")
print("\n--- Khởi tạo Datasets và DataLoaders cho Train, Dev, Test ---")
# Tạo Dataset và DataLoader
datasets = {}
dataloaders = {}
for split in ['train', 'dev', 'test']:
    print(f"\n=> Đang tạo Dataset cho split: {split}")
    datasets[split] = AISHELL1Dataset(
        AISHELL_TRANSCRIPT_PATH, AISHELL_WAV_ROOT, split=split,
        sample_rate=SAMPLE_RATE, n_mels=N_MELS,
        frame_length=FRAME_LENGTH, frame_shift=FRAME_SHIFT
    )
    if len(datasets[split]) == 0:
        print(f"!!! Dataset cho split '{split}' bị rỗng. Kiểm tra lại đường dẫn và file.")
        continue

    print(f"=> Đang tạo DataLoader cho split: {split}")
    # Shuffle=True cho tập train, False cho dev và test
    shuffle_data = (split == 'train')
    dataloaders[split] = DataLoader(
        datasets[split], batch_size=BATCH_SIZE,
        shuffle=shuffle_data, collate_fn=collate_fn,
        num_workers=NUM_WORKERS
    )
    print(f"Số lượng mẫu trong dataset '{split}': {len(datasets[split])}")
    print(f"Số lượng batch trong dataloader '{split}': {len(dataloaders[split])}")

if not dataloaders:
    exit("Không thể tạo bất kỳ DataLoader nào")

Sử dụng thiết bị: cuda

--- Khởi tạo Datasets và DataLoaders cho Train, Dev, Test ---

=> Đang tạo Dataset cho split: train
Initializing Dataset for SPLIT='train' with FBank params: n_mels=80, frame_length=25ms, frame_shift=10ms
Đang tìm kiếm file wav trong thư mục của split 'train': data_aishell\wav\train
Không tìm thấy thư mục con 'train' tại data_aishell\wav\train
!!! Dataset cho split 'train' bị rỗng. Kiểm tra lại đường dẫn và file.

=> Đang tạo Dataset cho split: dev
Initializing Dataset for SPLIT='dev' with FBank params: n_mels=80, frame_length=25ms, frame_shift=10ms
Đang tìm kiếm file wav trong thư mục của split 'dev': data_aishell\wav\dev
Không tìm thấy thư mục con 'dev' tại data_aishell\wav\dev
!!! Dataset cho split 'dev' bị rỗng. Kiểm tra lại đường dẫn và file.

=> Đang tạo Dataset cho split: test
Initializing Dataset for SPLIT='test' with FBank params: n_mels=80, frame_length=25ms, frame_shift=10ms
Đang tìm kiếm file wav trong thư mục của split 'test': data_aishell\wav\test


## Model

In [1]:
%cd ../

c:\Users\huy\OneDrive\Desktop\BTTH-DS313\asr_model_project


In [2]:
from src.asr_model import ASRModel

a, b, c = ASRModel(model_dim=768, mode = 'A').to('cuda'), ASRModel(model_dim=768, mode = 'B').to('cuda'), ASRModel(model_dim=768, mode = 'C').to('cuda')
a.params, b.params, c.params

c:\Users\huy\anaconda3\envs\vllm_env\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


(149466760, 149466760, 141578888)

In [3]:
import torch
audio_features = torch.randn(10, 20, 768).to('cuda')  # Example input tensor (seq_len, batch_size, acoustic_input_dim)
input_ids = torch.randint(0, 21128, (10, 35)).long().to('cuda')
attention_mask = torch.ones(10, 35).to('cuda')  # Example attention mask (batch_size, seq_len)

In [4]:
input_ids.shape, audio_features.shape, attention_mask.shape

(torch.Size([10, 35]), torch.Size([10, 20, 768]), torch.Size([10, 35]))

In [5]:
a_lala = a(input_ids = input_ids,
           attention_mask = None,
           audio_features = audio_features)
print(a_lala.shape)

torch.Size([10, 35, 21128])


In [6]:
b_lala = b(input_ids = input_ids,
           attention_mask = None,
           audio_features = audio_features)
print(b_lala.shape)

torch.Size([10, 35, 21128])


In [7]:
c_lala = c(input_ids = input_ids,
           attention_mask = None,
           audio_features = audio_features)
print(c_lala.shape)

torch.Size([10, 35, 21128])
